<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Download-images" data-toc-modified-id="Download-images-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Download images</a></span></li><li><span><a href="#Embed-faces" data-toc-modified-id="Embed-faces-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Embed faces</a></span></li><li><span><a href="#Download-and-embed" data-toc-modified-id="Download-and-embed-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download and embed</a></span></li></ul></div>

In [2]:
from query.datasets.prelude import *
from query.datasets.ingest import ingest_pose
import pyspark.sql.functions as func

import query.datasets.tvnews.embed_google_images as embed_google_images

# Download images

In [3]:
IMG_DOWNLOAD_DIR = '/app/data/google_images'

In [ ]:
curated_hosts = {
  "Hannity": {
    "producers": [],
    "hosts": ["Sean Hannity"]
  },
  "Happening Now": {
    "producers": [],
    "hosts": ["John Scott", "Melissa Francis"]
  },
  "All In With Chris Hayes": {
    "producers": [],
    "hosts": ["Chris Hayes"]
  },
  "Shepard Smith Reporting": {
    "producers": [],
    "hosts": ["Shepard Smith"]
  },
  "CNN Newsroom With Poppy Harlow": {
    "producers": [],
    "hosts": ["Poppy Harlow"]
  },
  "On the Record With Greta Van Susteren": {
    "producers": [],
    "hosts": ["Greta Van Susteren"]
  },
  "The OReilly Factor": {
    "producers": ["Bill O'Reilly"],
    "hosts": ["Bill O'Reilly"]
  },
  "The Kelly File": {
    "producers": [],
    "hosts": ["Megyn Kelly"]
  },
  "CNN Newsroom With Carol Costello": {
    "producers": [],
    "hosts": ["Carol Costello"]
  },
  "Americas Newsroom With Bill Hemmer and Martha MacCallum": {
    "producers": [],
    "hosts": ["Bill Hemmer", "Martha MacCallum"]
  },
  "Americas News HQ": {
    "producers": [],
    "hosts": ["Uma Pemmaraju", "Shannon Beam", "Kelly Wright", "Eric Shawn", "Arthel Neville", "Janice Dean", "Elizabeth Prann", "Doug McKelway", "Molly Henneberg", "Peter Doocy", "Bryan Llenas"]
  },
  "The Last Word With Lawrence ODonnell": {
    "producers": [],
    "hosts": ["Lawrence O'Donnell"]
  },
  "Hardball With Chris Matthews": {
    "producers": ["Ray Herbert"],
    "hosts": ["Chris Matthews"]
  },
  "Morning Joe": {
    "producers": ["Alex Korson", "Mike Buczkiewicz"],
    "hosts": ["Joe Scarborough", "Mika Brzezinski", "Willie Geist"]
  },
  "The 11th Hour With Brian Williams": {
    "producers": ["Julie Morse", "Patrick Burkey"],
    "hosts": ["Brian Williams"]
  },
  "Early Start With John Berman and Christine Romans": {
    "producers": [],
    "hosts": ["Christine Romans", "John Berman"]
  },
  "Outnumbered": {
    "producers": [],
    "hosts": ["Harris Faulkner", "Sandra Smith"]
  },
  "The Place for Politics 2016": {
    "producers": [],
    "hosts": ["Ari Melber", "Frances Rivera"]
  },
  "CNN Tonight With Don Lemon": {
    "producers": [],
    "hosts": ["Don Lemon"]
  },
  "Your World With Neil Cavuto": {
    "producers": [],
    "hosts": ["Neil Cavuto"]
  },
  "Anderson Cooper 360": {
    "producers": ["Charles Moore"],
    "hosts": ["Anderson Cooper"]
  },
  "CNN Newsroom With Fredricka Whitfield": {
    "producers": [],
    "hosts": ["Fredricka Whitfield"]
  },
  "CNN Newsroom Live": {
    "producers": [],
    "hosts": ["Brooke Baldwin", "John Berman", "Victor Blackwell", "Ana Cabrera", "Poppy Harlow", "Christi Paul", "Fredricka Whitfield"]
  },
  "CNN Newsroom With Brooke Baldwin": {
    "producers": [],
    "hosts": ["Brooke Baldwin"]
  },
  "MTP Daily": {
    "producers": ["John Reiss"],
    "hosts": ["Chuck Todd"]
  },
  "Wolf": {
    "producers": [],
    "hosts": ["Wolf Blitzer"]
  },
  "MSNBC Live": {
    "producers": [],
    "hosts": ["Brian Williams", "Chris Jansing", "Stephanie Ruhle", "Ali Velshi", "Hallie Jackson", "Craig Melvin", "Katy Tur", "Richard Lui"]
  },
  "The Rachel Maddow Show": {
    "producers": ["Rob Katko", "Bill Wolff", "Cory Gnazzo"],
    "hosts": ["Rachel Maddow"]
  },
  "New Day": {
    "producers": ["Javier Morgado"],
    "hosts": ["Chris Cuomo", "Alisyn Camerota", "Victor Blackwell", "Christi Paul"]
  },
  "The Lead With Jake Tapper": {
    "producers": ["Federico S. Quadrani"],
    "hosts": ["Jake Tapper"]
  },
  "Americas Newsroom": {
    "producers": [],
    "hosts": ["Bill Hemmer", "Sandra Smith"]
  },
  "Andrea Mitchell Reports": {
    "producers": [],
    "hosts": ["Andrea Mitchell"]
  },
  "The Five": {
    "producers": [],
    "hosts": ["Jesse Watters", "Kimberly Guilfoyle", "Greg Gutfeld", "Dana Perino", "Juan Williams"]
  },
  "Justice With Judge Jeanine": {
    "producers": [],
    "hosts": ["Jeanine Pirro"]
  },
  "FOX  Friends": {
    "producers": [],
    "hosts": ["Steve Doocy", "Ainsley Earhardt", "Brian Kilmeade", "Pete Hegseth", "Abby Huntsman", "Griff Jenkins"]
  },
  "Situation Room With Wolf Blitzer": {
    "producers": [],
    "hosts": ["Wolf Blitzer"]
  },
  "Erin Burnett OutFront": {
    "producers": [],
    "hosts": ["Erin Burnett"]
  },
  "Tucker Carlson Tonight": {
    "producers": [],
    "hosts": ["Tucker Carlson"]
  },
  "Special Report With Bret Baier": {
    "producers": [],
    "hosts": ["Bret Baier"]
  }
}

In [ ]:
for k, v in curated_hosts.items():
    for host in v['hosts']:
        print('Downloading images for:', host)
        embed_google_images.fetch_images(host)

# Embed faces

In [ ]:
EMBEDDINGS_DIR = '/app/data/face_embeddings'

In [ ]:
import _pickle as pickle

# Embed the hosts
for show, data in curated_hosts.items():
    for host in data['hosts']:
        print ('Generating embedding:', host)
        emb = embed_google_images.embed_directory(os.path.join(IMG_DOWNLOAD_DIR, host))
        if emb is None:
            print('Failed to embed', host, file=sys.stderr)
        else:
            emb_file = os.path.join(EMBEDDINGS_DIR, '%s.pkl' % host)
            with open(emb_file, 'wb') as f:
                pickle.dump(emb, f, protocol=2)

# Download and embed

In [ ]:
anderson_cooper = embed_google_images.name_to_embedding('anderson cooper')
john_mccain = embed_google_images.name_to_embedding('John McCain')

In [ ]:
import _pickle as pickle

def read_face_feature_file(file_name):
    with open(file_name, 'rb') as f:
        obj = pickle.load(f, encoding='latin1')
        faces = obj['faces']
        show_name = obj['name']
        show_id = obj['id']
        face_ids = []
        face_features = []
        face_id_to_meta = {}
        for x in faces:
            face_ids.append(x[0])
            face_features.append(x[1])
            face_id_to_meta[x[0]] = x[2]
        face_ids = np.array(face_ids, dtype=np.int32)
        face_features = np.stack(face_features, axis=0)
    return show_id, face_ids, face_features, face_id_to_meta

def get_face_features(video_id):
    return read_face_feature_file('/app/data/face_features_by_video/%d.pkl' % video_id)
    
ac360_videos = list(Video.objects.filter(show__name='Anderson Cooper 360'))
l2_dists = []
for video in ac360_videos:
    try:
        _, face_ids, face_features, _ = get_face_features(video.id)
        dists = np.linalg.norm(face_features - john_mccain, axis=1)
#         print(john_mccain.shape, dists.shape)
        for i in range(len(face_ids)):
            l2_dists.append((face_ids[i], dists[i]))
    except Exception as e:
        print(e)

In [ ]:
sorted_l2_dists = sorted(l2_dists, key=lambda x: x[1])
print(sorted_l2_dists[:100])

import cv2

def face_img(face):
    return crop(load_frame(face.person.frame.video, face.person.frame.number, []), face)

def faces_to_tiled_img(faces):
    def tile(imgs, rows=None, cols=None):
        # If neither rows/cols is specified, make a square
        if rows is None and cols is None:
            rows = int(math.sqrt(len(imgs)))

        if rows is None:
            rows = int((len(imgs) + cols - 1) / cols)
        else:
            cols = int((len(imgs) + rows - 1) / rows)

        # Pad missing frames with black
        diff = rows * cols - len(imgs)
        if diff != 0:
            imgs.extend([np.zeros(imgs[0].shape, dtype=imgs[0].dtype) for _ in range(diff)])

        return np.vstack([np.hstack(imgs[i * cols:(i + 1) * cols]) for i in range(rows)])
    
    face_imgs = par_for(face_img, faces)
    im = tile([cv2.resize(img, (100, 100)) for img in face_imgs])
    return im

im = faces_to_tiled_img(Face.objects.filter(id__in=[x[0] for x in sorted_l2_dists[:100]]))
imshow(im)
plt.show()
